# Unitary ansatz entering the VQE

The final energy output of a VQE calculation will crucially depend on the ansatz/form of the parameterized unitary $\hat U(\boldsymbol{\theta})$ employed in state preparation. Here we review two popular approaches, the unitary coupled cluster and qubit coupled cluster methodologies, and benchmark them for energy calculations of small molecules.

In [1]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients

## Unitary Coupled Cluster (UCC)

The UCC ansatz is obtained by 'unitarizing' the traditional coupled cluster ansatz,
$$ e^{\hat T} \rightarrow e^{\hat T - \hat T^\dagger} \equiv \hat U_{\text{UCC}}$$


Due to non-commutativity of terms in $\hat T - \hat T^\dagger$, the UCC ansatz does not have a straightforward decomposition in terms of circuit primitives implementable on the quantum computer. Therefore, to obtain a form which can be compiled, we employ the Trotter approximation. The accuracy of the circuit ansatz relative to the exact UCC operator will be dependent on how many Trotter steps are employed. The number of Trotter steps is commonly set to its minimal value of one to avoid excessive circuit depth.

In [2]:
trotter_steps = 1

### H2 in STO-3G basis

Below is a sample VQE simulation using the UCCSD ansatz compiled using a single trotter step for H$_2$ in minimal basis at $R=2.5$ (Angstrom). For comparison, we can run FCI to obtain the true ground state energy.


In [3]:
xyz_data = get_molecular_data('h4', geometry=2.5, xyz_format=True)
basis='sto-3g'

h4 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis, backend="pyscf")

print('Number of spin-orbitals (qubits): {} \n'.format(2*h4.n_orbitals))

E_FCI = h4.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

converged SCF energy = -1.32404666569946
Number of spin-orbitals (qubits): 8 

FCI energy: -1.8731741642964097


The UCCSD VQE optimization is of the form
$$E = \min_{\boldsymbol{\theta}} \langle \text{HF} | \hat U_{\text{UCC}}^\dagger(\boldsymbol{\theta}) \hat H  \hat U_{\text{UCC}} (\boldsymbol{\theta}) | \text{HF} \rangle $$
To expedite the optimization process, we can set the initial guess of the amplitudes to zero, i.e. the optimization will begin at the Hartree Fock state. This heuristic is best suited for when Hartree Fock is believed to be in qualitative agreement with the true ground state. To further alleviate quantum resources, we can estimate the amplitudes using classical electronic structure methods (here, MP2 perturbation theory), and only include the unitaries with non-zero estimated amplitudes.

In [4]:
H = h4.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h4.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-6)

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 105 terms


Number of UCCSD amplitudes: 8 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
device          : None
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 24 expectationvalues

active variables : 8

E=-1.32404667  angles= {(3, 1, 3, 1): 0.0, (2, 0, 2, 0): 0.0, (3, 0, 3, 0): 0.0, (2, 1, 2, 1): 0.0, ((0, 2), 'S', None): 0.0, ((0, 3), 'S', None): 0.0, ((1, 2), 'S', None): 0.0, ((1, 3), 'S', None): 0.0}  samples= None
E=-1.66110302  angles= {(3, 1, 3, 1): -0.4807424545288086, (2, 0, 2, 0): -0.4654768705368042, (3, 0, 3, 0): -0.18619823455810547, (2, 1, 2, 1): -0.0004909038543701172, ((0, 2), 'S', None): 0.0, ((0, 3), 'S', None): 0.0, ((1, 2), 'S', None): 0.0, ((1, 3), 'S', None): 0.0}  samples= None
E=-1.74352446  angles= {(3, 1, 3, 1): -0.9406380249859474, (2, 0, 2, 0): -0.9083609536962874, (

We see that the converged UCCSD energy is in exact agreement with the FCI energy, as expected for a $2$-electron system. 

### H2O in 6-31G basis

Now let us try a larger problem of H$_2$O in 6-31G basis. However, we will restrict the active space. The unrestricted problem leads to a $14$-qubit Hamiltonian, and $34$ UCCSD amplitudes to optimize even after neglecting the zero MP2 amplitudes. Therefore, we will remove some orbital degrees of freedom which are less important in accurately describing the electronic structure. By freezing all orbitals other than $0b_1$, $1b_1$, $2a_1$, and $3a_1$, we reduce the problem to an $8$-qubit Hamiltonian with $8$ UCCSD variational amplitudes.## 6-31G basis

In [5]:
xyz_data = get_molecular_data('h4', geometry=1, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
h4 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, backend="pyscf")

print('Number of spin-orbitals (qubits): {} \n'.format(2*h4.n_orbitals))

E_FCI = h4.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

converged SCF energy = -1.88376628204839
Number of spin-orbitals (qubits): 16 

FCI energy: -2.0317569083334117


We will then run the UCCSD VQE simulation (***warning: tq.minimize will take several minutes - 1 hour + to converge for a VQE instance of this size.*** Smaller active spaces can be employed to lower VQE simulation runtimes).

In [ ]:
H = h4.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h4.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4)

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 1617 terms


Number of UCCSD amplitudes: 52 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
device          : None
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 224 expectationvalues

active variables : 52

E=-1.88376628  angles= {(2, 0, 2, 0): 0.0, (3, 1, 3, 1): 0.0, (6, 1, 6, 1): 0.0, (2, 1, 2, 1): 0.0, (5, 0, 2, 0): 0.0, (2, 0, 5, 0): 0.0, (7, 1, 7, 1): 0.0, (5, 0, 5, 0): 0.0, (3, 1, 7, 1): 0.0, (7, 1, 3, 1): 0.0, (6, 1, 7, 0): 0.0, (7, 1, 6, 0): 0.0, (7, 0, 6, 1): 0.0, (6, 0, 7, 1): 0.0, (4, 1, 6, 0): 0.0, (6, 1, 4, 0): 0.0, (6, 0, 4, 1): 0.0, (4, 0, 6, 1): 0.0, (4, 0, 4, 0): 0.0, (6, 0, 6, 0): 0.0, (7, 0, 7, 0): 0.0, (3, 0, 3, 0): 0.0, (3, 0, 6, 1): 0.0, (6, 0, 3, 1): 0.0, (6, 1, 3, 0): 0.0, (3, 1, 6, 0): 0.0, (3, 0, 7, 0): 0.0, (7, 0, 3, 0): 0.0, (4, 1, 4, 1): 0.0, (4, 1, 7, 1): 0.0, (7,

E=-1.86521410  angles= {(2, 0, 2, 0): -0.03957330663828364, (3, 1, 3, 1): -0.024923609530376763, (6, 1, 6, 1): -0.02625536938936422, (2, 1, 2, 1): -0.004170974607165073, (5, 0, 2, 0): 0.018218021461420528, (2, 0, 5, 0): 0.01822013702137607, (7, 1, 7, 1): -0.014212395325581398, (5, 0, 5, 0): -0.010694015344491888, (3, 1, 7, 1): -0.012327978604675084, (7, 1, 3, 1): -0.01232783970748599, (6, 1, 7, 0): -0.01406874103161056, (7, 1, 6, 0): -0.009996656185980064, (7, 0, 6, 1): -0.01407611415936108, (6, 0, 7, 1): -0.009987557453732072, (4, 1, 6, 0): -0.002525389944651048, (6, 1, 4, 0): 0.021311826903561974, (6, 0, 4, 1): -0.0025213861113381753, (4, 0, 6, 1): 0.021300877744733056, (4, 0, 4, 0): -0.009347341343954783, (6, 0, 6, 0): -0.010281594215427337, (7, 0, 7, 0): -0.007113958689887655, (3, 0, 3, 0): -0.007989762199374893, (3, 0, 6, 1): -0.015927518773275116, (6, 0, 3, 1): 0.0004326934891250092, (6, 1, 3, 0): -0.015930393295994938, (3, 1, 6, 0): 0.0004349972880600978, (3, 0, 7, 0): -0.007250

E=-1.91635584  angles= {(2, 0, 2, 0): -0.12082912341574821, (3, 1, 3, 1): -0.10382615183072061, (6, 1, 6, 1): -0.04174100178624825, (2, 1, 2, 1): 0.01690131906055778, (5, 0, 2, 0): 0.04345736632659958, (2, 0, 5, 0): 0.04347899809196772, (7, 1, 7, 1): -0.04641744218608783, (5, 0, 5, 0): -0.036466202045939304, (3, 1, 7, 1): -0.03125571060896729, (7, 1, 3, 1): -0.03127601039173642, (6, 1, 7, 0): -0.04827655908219424, (7, 1, 6, 0): -0.047784060124679714, (7, 0, 6, 1): -0.048267068241444805, (6, 0, 7, 1): -0.04780528523789564, (4, 1, 6, 0): 0.022160568342836515, (6, 1, 4, 0): 0.022300066701449288, (6, 0, 4, 1): 0.022139121915607227, (4, 0, 6, 1): 0.02236869414303145, (4, 0, 4, 0): -0.030991608756100495, (6, 0, 6, 0): -0.02748588092022728, (7, 0, 7, 0): 0.01233638022385252, (3, 0, 3, 0): 0.003668194450125673, (3, 0, 6, 1): -0.00962508593031827, (6, 0, 3, 1): -0.011461425998825775, (6, 1, 3, 0): -0.009617254278581039, (3, 1, 6, 0): -0.011468571246069264, (3, 0, 7, 0): -0.027895179092958997, (

E=-1.93293265  angles= {(2, 0, 2, 0): -0.11096179630673829, (3, 1, 3, 1): -0.08508340266004039, (6, 1, 6, 1): -0.050437213953902924, (2, 1, 2, 1): 0.015928857285652114, (5, 0, 2, 0): 0.04080784482858561, (2, 0, 5, 0): 0.04083628605854513, (7, 1, 7, 1): -0.026330571352704044, (5, 0, 5, 0): -0.022266868010060514, (3, 1, 7, 1): -0.030871663623334183, (7, 1, 3, 1): -0.030879548884324717, (6, 1, 7, 0): -0.02174207396798995, (7, 1, 6, 0): -0.023052206288947642, (7, 0, 6, 1): -0.021743175153756723, (6, 0, 7, 1): -0.023050825396983166, (4, 1, 6, 0): 0.01638809889747415, (6, 1, 4, 0): 0.019177712314338482, (6, 0, 4, 1): 0.016391745100838944, (4, 0, 6, 1): 0.019174153832603336, (4, 0, 4, 0): -0.017113265195931274, (6, 0, 6, 0): -0.01871748537995546, (7, 0, 7, 0): -0.016890499341416426, (3, 0, 3, 0): -0.010817719926941003, (3, 0, 6, 1): -0.015703803314187928, (6, 0, 3, 1): -0.017370163852589576, (6, 1, 3, 0): -0.015706581760424002, (3, 1, 6, 0): -0.01736813505843058, (3, 0, 7, 0): -0.011705151457

E=-1.93368625  angles= {(2, 0, 2, 0): -0.12511219137389593, (3, 1, 3, 1): -0.09387346498532935, (6, 1, 6, 1): -0.048548491929395664, (2, 1, 2, 1): 0.08573759409937454, (5, 0, 2, 0): 0.04097097913524858, (2, 0, 5, 0): 0.04099830681152979, (7, 1, 7, 1): -0.028418055537518302, (5, 0, 5, 0): -0.020799153068862004, (3, 1, 7, 1): -0.033024706045737304, (7, 1, 3, 1): -0.033040304028913864, (6, 1, 7, 0): -0.021185603747895687, (7, 1, 6, 0): -0.022482464026674424, (7, 0, 6, 1): -0.021187754271902528, (6, 0, 7, 1): -0.022479965577166094, (4, 1, 6, 0): 0.016088427731419632, (6, 1, 4, 0): 0.018840514329397515, (6, 0, 4, 1): 0.01608974540592285, (4, 0, 6, 1): 0.018840711542882846, (4, 0, 4, 0): -0.016591248017564952, (6, 0, 6, 0): -0.01868573988687815, (7, 0, 7, 0): -0.01595177457811738, (3, 0, 3, 0): -0.007395787906741367, (3, 0, 6, 1): -0.015493403633864049, (6, 0, 3, 1): -0.01710736267362383, (6, 1, 3, 0): -0.015495536799184816, (3, 1, 6, 0): -0.017105031195313994, (3, 0, 7, 0): -0.0103902744937

E=-1.93420491  angles= {(2, 0, 2, 0): -0.13601904096781803, (3, 1, 3, 1): -0.10122006806986272, (6, 1, 6, 1): -0.048582465511127014, (2, 1, 2, 1): 0.22829575503408817, (5, 0, 2, 0): 0.04185751857740929, (2, 0, 5, 0): 0.04188209083966271, (7, 1, 7, 1): -0.029185384692776085, (5, 0, 5, 0): -0.019175727461054885, (3, 1, 7, 1): -0.03413087578993157, (7, 1, 3, 1): -0.034137384118773234, (6, 1, 7, 0): -0.02247358313242137, (7, 1, 6, 0): -0.023837535137670497, (7, 0, 6, 1): -0.022488721930407258, (6, 0, 7, 1): -0.023820009127934007, (4, 1, 6, 0): 0.015511567409116718, (6, 1, 4, 0): 0.018271089443233062, (6, 0, 4, 1): 0.015509631220550652, (4, 0, 6, 1): 0.018278153278858855, (4, 0, 4, 0): -0.016322357213411895, (6, 0, 6, 0): -0.01876195016849442, (7, 0, 7, 0): -0.01606912316844941, (3, 0, 3, 0): -0.005916906684491386, (3, 0, 6, 1): -0.015619182950699613, (6, 0, 3, 1): -0.017153973971541756, (6, 1, 3, 0): -0.015630515581943734, (3, 1, 6, 0): -0.017148614525014576, (3, 0, 7, 0): -0.0082325805433

E=-1.93433642  angles= {(2, 0, 2, 0): -0.13472289864579126, (3, 1, 3, 1): -0.10161343076865188, (6, 1, 6, 1): -0.047369305684943415, (2, 1, 2, 1): 0.23560902227359218, (5, 0, 2, 0): 0.04118948016134906, (2, 0, 5, 0): 0.04121481407715348, (7, 1, 7, 1): -0.030529794274055392, (5, 0, 5, 0): -0.02011083258468934, (3, 1, 7, 1): -0.03669006802598727, (7, 1, 3, 1): -0.036708843113708305, (6, 1, 7, 0): -0.021267004934856237, (7, 1, 6, 0): -0.02257039622552749, (7, 0, 6, 1): -0.021267081990536185, (6, 0, 7, 1): -0.022570074562545936, (4, 1, 6, 0): 0.015827204743657786, (6, 1, 4, 0): 0.018623832385301138, (6, 0, 4, 1): 0.015832240972466324, (4, 0, 6, 1): 0.01861858855455046, (4, 0, 4, 0): -0.016601377704871315, (6, 0, 6, 0): -0.018115952192888394, (7, 0, 7, 0): -0.015887043982132202, (3, 0, 3, 0): -0.004962651403661206, (3, 0, 6, 1): -0.015494214689701292, (6, 0, 3, 1): -0.017009424716936714, (6, 1, 3, 0): -0.015495816973662471, (3, 1, 6, 0): -0.017008008195800752, (3, 0, 7, 0): -0.0110485956398

E=-1.93433724  angles= {(2, 0, 2, 0): -0.13486451330690724, (3, 1, 3, 1): -0.10142582654489402, (6, 1, 6, 1): -0.04726497034881316, (2, 1, 2, 1): 0.23940474121964797, (5, 0, 2, 0): 0.04109380156094503, (2, 0, 5, 0): 0.04111994640710464, (7, 1, 7, 1): -0.030629276481763983, (5, 0, 5, 0): -0.02027242272837691, (3, 1, 7, 1): -0.036816127999830174, (7, 1, 3, 1): -0.03683523567393526, (6, 1, 7, 0): -0.021284863143176323, (7, 1, 6, 0): -0.022587066084543864, (7, 0, 6, 1): -0.02128462182451337, (6, 0, 7, 1): -0.02258809317098136, (4, 1, 6, 0): 0.0157740836801097, (6, 1, 4, 0): 0.01856937611595234, (6, 0, 4, 1): 0.015776605085043402, (4, 0, 6, 1): 0.01856675602759828, (4, 0, 4, 0): -0.016634430115860802, (6, 0, 6, 0): -0.01806386128014409, (7, 0, 7, 0): -0.015892822034453708, (3, 0, 3, 0): -0.005046505645063193, (3, 0, 6, 1): -0.015593677690133343, (6, 0, 3, 1): -0.01710788766648266, (6, 1, 3, 0): -0.015595412522919233, (3, 1, 6, 0): -0.017106131289074414, (3, 0, 7, 0): -0.011091200468523816, 

## QCC
### H4 in STO-3G basis
Below we perform the entangler screening protocol for H4 in minimal basis, and obtain one grouping of entanglers with non-zero energy gradient. We then select one of them to be used in the QCC VQE simulation.

In [ ]:
xyz_data = get_molecular_data('h4', geometry=2.5, xyz_format=True)
basis='sto-3g'

h4 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set='sto-3g', backend="pyscf")

hf_reference = hf_occ(2*h4.n_orbitals, h4.n_electrons)

H = h4.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 1

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*h4.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h4.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

### VQE optimization for ansatz

In [ ]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*h4.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-6)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

## 6-31G basis

In [ ]:
xyz_data = get_molecular_data('h4', geometry=1, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
h4 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, backend="pyscf")
hf_reference = hf_occ(2*h4.n_orbitals, h4.n_electrons)


H = h4.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 6

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*h4.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h4.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

In [ ]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*h4.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4)


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

In [ ]:
n = 10


result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result))